<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/DeepSEA_beluga_getridofpermutelayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://mitra.stanford.edu/kundaje/avanti/deepsea_beluga/deepseabeluga_keras.h5 -O deepseabeluga_keras.h5

--2019-10-08 18:21:31--  http://mitra.stanford.edu/kundaje/avanti/deepsea_beluga/deepseabeluga_keras.h5
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 598195356 (570M)
Saving to: ‘deepseabeluga_keras.h5’

deepseabeluga_keras 100%[===================>] 570.48M  21.4MB/s    in 35s     

2019-10-08 18:22:07 (16.2 MB/s) - ‘deepseabeluga_keras.h5’ saved [598195356/598195356]



In [1]:
import keras
from keras.models import load_model

model = load_model("deepseabeluga_keras.h5")
model.summary()

Using TensorFlow backend.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
___

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
from keras.models import Sequential
from keras.layers.core import Dense,Flatten
from keras.layers.convolutional import Conv1D

new_model = Sequential()
#Different first layer with input_shape=(400,4) instead of (2000,4)
new_model.add(Conv1D(kernel_size=8, filters=320, input_shape=(2000,4)))
new_model.layers[0].set_weights(model.layers[0].get_weights())
#We copy over remaining layers up until the Permute layer, which we are
# going to get rid of the need for (I had the Permute layer to cope
# with the fact that the original pytorch model was in a 'channels first'
# format, but Keras is 'channels last', and the weights of the pytorch Dense
# layer were such that it was expecting the output of the conv layer to
# be in a 'channels first' format - however, we will deal with this by
# just rearranging the weights of the Dense layer, since we have to modify
# those weights anyway).
for layer in model.layers[1:16]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  if (len(layer.get_weights()) > 0):
      new_layer.build(layer.input_shape)
  new_layer.set_weights(layer.get_weights())
  new_model.add(new_layer)
new_model.add(Flatten())
new_model.add(Dense(2003,name="altereddense"))
for layer in model.layers[20:]:
  layer_config = layer.get_config()
  del layer_config['name'] #let the name be autogenerated to avoid collisions
  new_layer = type(layer)(**layer_config)
  if (len(layer.get_weights()) > 0):
      new_layer.build(layer.input_shape)
  new_layer.set_weights(layer.get_weights())
  new_model.add(new_layer)

first_dense_layer_weights, first_dense_layer_biases = model.layers[19].get_weights()
#These next lines alter the weights of the dense layer so that the whole model
# will work on an input of length 400bp
#While porting the weights over, we also get rid of the need for the Permute
# layer by applying the 'transpose' on the weights. The truncation of 50:56 is
# what gets the model to work on an input of length 400bp
new_first_dense_layer_weights = first_dense_layer_weights.reshape((640,106,2003))[:,:,:].transpose((1,0,2)).reshape((640*106,2003))
new_model.layers[17].set_weights([new_first_dense_layer_weights, first_dense_layer_biases])

In [0]:
import numpy as np
random_inputs = np.random.RandomState(1).rand(20,2000,4)

In [0]:
new_preds = new_model.predict(random_inputs[:,:,:])

In [0]:
old_preds = model.predict(random_inputs[:,:,:])

In [10]:
old_preds

array([[0.01404554, 0.00812003, 0.03995261, ..., 0.00140196, 0.00124151,
        0.00027382],
       [0.01368925, 0.00793654, 0.03913385, ..., 0.00138   , 0.00120762,
        0.000267  ],
       [0.01411042, 0.00824639, 0.04028752, ..., 0.00141224, 0.00125223,
        0.00027698],
       ...,
       [0.01376671, 0.00773171, 0.03907946, ..., 0.00139257, 0.00119781,
        0.00026673],
       [0.01378527, 0.00781977, 0.0391092 , ..., 0.00137195, 0.00119868,
        0.00026551],
       [0.01403785, 0.00804397, 0.03988519, ..., 0.00137761, 0.00121772,
        0.00026935]], dtype=float32)

In [11]:
new_preds

array([[0.01404554, 0.00811997, 0.03995261, ..., 0.0014019 , 0.00124151,
        0.00027376],
       [0.01368931, 0.00793654, 0.03913385, ..., 0.00138006, 0.00120765,
        0.000267  ],
       [0.01411042, 0.00824639, 0.04028746, ..., 0.00141224, 0.00125223,
        0.00027704],
       ...,
       [0.01376674, 0.00773171, 0.03907955, ..., 0.00139257, 0.00119784,
        0.00026673],
       [0.01378524, 0.00781977, 0.03910929, ..., 0.00137201, 0.00119865,
        0.00026539],
       [0.01403782, 0.00804397, 0.03988519, ..., 0.00137761, 0.00121778,
        0.00026935]], dtype=float32)

In [7]:
np.max(np.abs(old_preds-new_preds))

2.682209e-07

In [9]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 489, 480)         

In [0]:
new_model.save("deepseabeluga_keras_nopermutelayer.h5")